In [1]:
import pandas as pd
import numpy as np
import requests
import re
import nltk
import spacy
from nltk.corpus import stopwords
import string

/home/long/anaconda3/envs/mlops-env/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
df = pd.read_csv('../data/process_v1/process_data_4.csv')

In [23]:
df

,numberOfFloors,numberOfBathRooms,numberOfBedRooms,numberOfKitchens,numberOfGarages,certificateOfLandUseRight,ward,street,district,city,...,frontWidth,endWidth,facade,houseDirection,landSize,price,unitPrice,accessibility,landType,description
0,1,NaN,NaN,NaN,NaN,no,hoài đức - vân canh,hoài đức - vân canh - hậu ái,hoài đức,hà nội,...,4.2,NaN,oneSideOpen,southeast,50.0,4.60,billion,fitThreeCars,residentialLand,"bán đất dịch vụ 6,9ha vân canh, hoài đức, hà n..."
1,5,5.0,4.0,NaN,NaN,yes,hoàng mai - giáp bát,hoàng mai - giáp bát - giáp bát,hoàng mai,hà nội,...,0.0,NaN,oneSideOpen,NaN,35.0,3.60,billion,fitOneCarAndOneMotorbike,residentialLand,"+ cần bán nhà phố giáp bát, giáp tuyến phố kim..."
2,4,NaN,NaN,NaN,NaN,yes,hà đông - văn quán,hà đông - văn quán - chiến thắng,hà đông,hà nội,...,4.5,4.5,oneSideOpen,NaN,40.0,3.90,billion,notInTheAlley,residentialLand,"bán nhà giá bất ngờ 3.9 tỷ,có dt chính 40 m2 ,..."
3,7,7.0,7.0,NaN,NaN,yes,hoàn kiếm - lý thái tổ,hoàn kiếm - lý thái tổ - lý thái tổ,hoàn kiếm,hà nội,...,7.0,NaN,twoSideOpen,NaN,120.0,165.00,billion,notInTheAlley,residentialLand,cần bán tòa văn phòng kết hợp phòng ở tầng trê...
4,4,NaN,NaN,NaN,NaN,yes,cầu giấy - nghĩa đô,cầu giấy - nghĩa đô - hoàng quốc việt,cầu giấy,hà nội,...,0.0,NaN,twoSideOpen,NaN,50.0,26.70,billion,notInTheAlley,residentialLand,"+ nhà 4 tầng, kinh doanh hàng ăn, phố kinh doa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170959,1,2.0,2.0,NaN,NaN,no,hai bà trưng - vĩnh tuy,hai bà trưng - vĩnh tuy - mạc thị bưởi,hai bà trưng,hà nội,...,0.0,NaN,oneSideOpen,NaN,70.0,2.70,billion,notInTheAlley,residentialLand,chủ nhà cần bán căn hộ với thông tin chính xác...
170960,6,10.0,10.0,NaN,NaN,no,thanh xuân - thượng đình,thanh xuân - thượng đình - thượng đình,thanh xuân,hà nội,...,4.5,NaN,oneSideOpen,NaN,45.0,6.99,billion,narrorRoad,residentialLand,bán nhà đường khương đình thanh xuân kinh doan...
170961,8,22.0,22.0,NaN,NaN,yes,long biên - long biên,long biên - long biên - cổ linh,long biên,hà nội,...,4.3,NaN,twoSideOpen,southwest,82.0,25.00,billion,fitTwoCars,residentialLand,anh chị nhà phố cổ cần bán tòa khách sạn doanh...
170962,5,NaN,NaN,NaN,NaN,yes,nam từ liêm - mỹ đình 1,nam từ liêm - mỹ đình 1 - đình thôn,nam từ liêm,hà nội,...,0.0,NaN,oneSideOpen,NaN,69.0,10.90,billion,notInTheAlley,residentialLand,"cần ngay vốn, bán nhà vị trí mặt tiền tọa lạc ..."


In [3]:
s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
def remove_accents(input_str):
	s = ''
	for c in input_str:
		if c in s1:
			s += s0[s1.index(c)]
		else:
			s += c
	return s

streets = [remove_accents(item) for item in df['street'].unique()]

In [4]:
address_words = []

for street in streets:
    address_words += street.split(" ")
address_words[0]

'hoai'

In [5]:
eng_1_df = pd.read_csv('../data/process_v1/trans/eng_1_20000.csv')
eng_2_df = pd.read_csv('../data/process_v1/trans/eng_2_20000.csv')
eng_3_df = pd.read_csv('../data/process_v1/trans/eng_3_20000.csv')
eng_4_df = pd.read_csv('../data/process_v1/trans/eng_4_20000.csv')
eng_5_df = pd.read_csv('../data/process_v1/trans/eng_5_20000.csv')
eng_6_df = pd.read_csv('../data/process_v1/trans/eng_6_20000.csv')
eng_7_df = pd.read_csv('../data/process_v1/trans/eng_7_20000.csv')
eng_8_df = pd.read_csv('../data/process_v1/trans/eng_8_27000.csv')

In [6]:
eng_df = pd.concat([eng_1_df, eng_2_df, eng_3_df, eng_4_df, eng_5_df,eng_6_df ,eng_7_df, eng_8_df])
eng_df

,en_description,description
0,Selling service land of 6.9 hectares in Van Ca...,"bán đất dịch vụ 6,9ha vân canh, hoài đức, hà n..."
1,"Need to sell a townhouse adjacent to bowl, adj...","+ cần bán nhà phố giáp bát, giáp tuyến phố kim..."
2,Selling a house for a surprising price of 3.9 ...,"bán nhà giá bất ngờ 3.9 tỷ,có dt chính 40 m2 ,..."
3,Need to sell office building with room on the ...,cần bán tòa văn phòng kết hợp phòng ở tầng trê...
4,"4-storey house, restaurant business, busy busi...","+ nhà 4 tầng, kinh doanh hàng ăn, phố kinh doa..."
...,...,...
26995,The owner needs capital to do a project abroad...,mặt phố bạch mai. khu vực kinh doanh đẳng cấp ...
26996,"The house is solidly built, dedicated to livin...",- bán tòa nhà hoàng hoa thám - q. bình thạnh. ...
26997,Fold for only 12.5 billion to own a house righ...,"bán nhà vườn lài, hxh thông, ngang 4.5x14m, ga..."
26998,"House on Nguyen Phuc Nguyen, Ward 10, District...","ra mắt 58 căn biệt thự nhà vườn đầu tiên, cạnh..."


In [7]:
eng_df['en_description'] = eng_df['en_description'].fillna("No information")

In [8]:
eng_df["text_lower"] = eng_df["en_description"].str.lower()
eng_df.head()

,en_description,description,text_lower
0,Selling service land of 6.9 hectares in Van Ca...,"bán đất dịch vụ 6,9ha vân canh, hoài đức, hà n...",selling service land of 6.9 hectares in van ca...
1,"Need to sell a townhouse adjacent to bowl, adj...","+ cần bán nhà phố giáp bát, giáp tuyến phố kim...","need to sell a townhouse adjacent to bowl, adj..."
2,Selling a house for a surprising price of 3.9 ...,"bán nhà giá bất ngờ 3.9 tỷ,có dt chính 40 m2 ,...",selling a house for a surprising price of 3.9 ...
3,Need to sell office building with room on the ...,cần bán tòa văn phòng kết hợp phòng ở tầng trê...,need to sell office building with room on the ...
4,"4-storey house, restaurant business, busy busi...","+ nhà 4 tầng, kinh doanh hàng ăn, phố kinh doa...","4-storey house, restaurant business, busy busi..."


In [9]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

eng_df["text_wo_punct"] = eng_df["text_lower"].apply(lambda text: remove_punctuation(text))
eng_df.head()

,en_description,description,text_lower,text_wo_punct
0,Selling service land of 6.9 hectares in Van Ca...,"bán đất dịch vụ 6,9ha vân canh, hoài đức, hà n...",selling service land of 6.9 hectares in van ca...,selling service land of 69 hectares in van can...
1,"Need to sell a townhouse adjacent to bowl, adj...","+ cần bán nhà phố giáp bát, giáp tuyến phố kim...","need to sell a townhouse adjacent to bowl, adj...",need to sell a townhouse adjacent to bowl adja...
2,Selling a house for a surprising price of 3.9 ...,"bán nhà giá bất ngờ 3.9 tỷ,có dt chính 40 m2 ,...",selling a house for a surprising price of 3.9 ...,selling a house for a surprising price of 39 b...
3,Need to sell office building with room on the ...,cần bán tòa văn phòng kết hợp phòng ở tầng trê...,need to sell office building with room on the ...,need to sell office building with room on the ...
4,"4-storey house, restaurant business, busy busi...","+ nhà 4 tầng, kinh doanh hàng ăn, phố kinh doa...","4-storey house, restaurant business, busy busi...",4storey house restaurant business busy busines...


In [10]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

eng_df["text_wo_stop"] = eng_df["text_wo_punct"].apply(lambda text: remove_stopwords(text))
eng_df.head()

,en_description,description,text_lower,text_wo_punct,text_wo_stop
0,Selling service land of 6.9 hectares in Van Ca...,"bán đất dịch vụ 6,9ha vân canh, hoài đức, hà n...",selling service land of 6.9 hectares in van ca...,selling service land of 69 hectares in van can...,selling service land 69 hectares van canh hoai...
1,"Need to sell a townhouse adjacent to bowl, adj...","+ cần bán nhà phố giáp bát, giáp tuyến phố kim...","need to sell a townhouse adjacent to bowl, adj...",need to sell a townhouse adjacent to bowl adja...,need sell townhouse adjacent bowl adjacent kim...
2,Selling a house for a surprising price of 3.9 ...,"bán nhà giá bất ngờ 3.9 tỷ,có dt chính 40 m2 ,...",selling a house for a surprising price of 3.9 ...,selling a house for a surprising price of 39 b...,selling house surprising price 39 billion main...
3,Need to sell office building with room on the ...,cần bán tòa văn phòng kết hợp phòng ở tầng trê...,need to sell office building with room on the ...,need to sell office building with room on the ...,need sell office building room upper floor fac...
4,"4-storey house, restaurant business, busy busi...","+ nhà 4 tầng, kinh doanh hàng ăn, phố kinh doa...","4-storey house, restaurant business, busy busi...",4storey house restaurant business busy busines...,4storey house restaurant business busy busines...


In [11]:
from collections import Counter
cnt = Counter()
for text in eng_df["text_wo_stop"].values:
    for word in text.split():
        cnt[word] += 1

In [12]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

eng_df["text_wo_stopfreq"] = eng_df["text_wo_stop"].apply(lambda text: remove_freqwords(text))
eng_df.head()

,en_description,description,text_lower,text_wo_punct,text_wo_stop,text_wo_stopfreq
0,Selling service land of 6.9 hectares in Van Ca...,"bán đất dịch vụ 6,9ha vân canh, hoài đức, hà n...",selling service land of 6.9 hectares in van ca...,selling service land of 69 hectares in van can...,selling service land 69 hectares van canh hoai...,selling service land 69 hectares van canh hoai...
1,"Need to sell a townhouse adjacent to bowl, adj...","+ cần bán nhà phố giáp bát, giáp tuyến phố kim...","need to sell a townhouse adjacent to bowl, adj...",need to sell a townhouse adjacent to bowl adja...,need sell townhouse adjacent bowl adjacent kim...,need sell townhouse adjacent bowl adjacent kim...
2,Selling a house for a surprising price of 3.9 ...,"bán nhà giá bất ngờ 3.9 tỷ,có dt chính 40 m2 ,...",selling a house for a surprising price of 3.9 ...,selling a house for a surprising price of 39 b...,selling house surprising price 39 billion main...,selling surprising 39 main ​​40 m2 beautiful l...
3,Need to sell office building with room on the ...,cần bán tòa văn phòng kết hợp phòng ở tầng trê...,need to sell office building with room on the ...,need to sell office building with room on the ...,need sell office building room upper floor fac...,need sell office building room upper facing ly...
4,"4-storey house, restaurant business, busy busi...","+ nhà 4 tầng, kinh doanh hàng ăn, phố kinh doa...","4-storey house, restaurant business, busy busi...",4storey house restaurant business busy busines...,4storey house restaurant business busy busines...,4storey restaurant business busy business rent...


In [13]:
len(cnt.most_common())

83711

In [14]:
n_rare_words = 40000
RAREWORDS = [w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]]
NUM_WORDS = [w for (w, wc) in cnt.most_common() if sum([_.isnumeric() for _ in w])]
ADDRESS_WORDS = [w for (w, wc) in cnt.most_common() if w in address_words]

RAREWORDS = set(RAREWORDS + NUM_WORDS + ADDRESS_WORDS)
print(RAREWORDS)

{'k🤼', 'dzin', '\u200b\u200b1405m2', '8mx205m', '1933', 'businessbrhouse', 'industrialbr', '1tumblr', '53x155m', '40150', '187m²', '0989894845', 's5013211', 'nuo', '238m²', 'ocenpak', 'stellas', '3135', '13714mm2', '300400m', '7224m²', 'wak', '0', 'roadbrme', 'q28', '8mx345m', 'walkwaybron', 'x2x3', '0968686285', '2100', 'tructrandautu79gmailcom', 'tb673323', '❤️☎️☎️', 'dtxd38m', 'consultationdivbrsupport', '0913918915', '0904545681div', '4126m²', '0975100', '1254m', '13267', '48m26t38mt68bn', '𝙱𝙰𝚈div', 'million120m', '2121', '175x27m', '31e', '0934511', 'nearp', 'dragging', '4door', '3x05', '0979266220', 'g3cd', '7974', '80mx5t', '0932323204', '3v7', 'solari', '5796', 'pbrgoods', '24355', 'hp06', '397m²', '25x28m', '0936117955', '1117', '0912800949div', '\u200b\u200boffering', 's48m²', '50m²4tapproximately', '35months', '80m2divdiv', '0899990683', 'marttay', 'billiondirection', 'datzaloviber', '35x23m', '053', 'footage66m', '371m2', '02th', 'schoolpparea', 'businessinvestmentcash', '1

In [15]:
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

eng_df["text_wo_stopfreqrare"] = eng_df["text_wo_stopfreq"].apply(lambda text: remove_rarewords(text))
eng_df.head()

,en_description,description,text_lower,text_wo_punct,text_wo_stop,text_wo_stopfreq,text_wo_stopfreqrare
0,Selling service land of 6.9 hectares in Van Ca...,"bán đất dịch vụ 6,9ha vân canh, hoài đức, hà n...",selling service land of 6.9 hectares in van ca...,selling service land of 69 hectares in van can...,selling service land 69 hectares van canh hoai...,selling service land 69 hectares van canh hoai...,selling service land hectares hanoi corner lot...
1,"Need to sell a townhouse adjacent to bowl, adj...","+ cần bán nhà phố giáp bát, giáp tuyến phố kim...","need to sell a townhouse adjacent to bowl, adj...",need to sell a townhouse adjacent to bowl adja...,need sell townhouse adjacent bowl adjacent kim...,need sell townhouse adjacent bowl adjacent kim...,need sell townhouse adjacent bowl adjacent hou...
2,Selling a house for a surprising price of 3.9 ...,"bán nhà giá bất ngờ 3.9 tỷ,có dt chính 40 m2 ,...",selling a house for a surprising price of 3.9 ...,selling a house for a surprising price of 39 b...,selling house surprising price 39 billion main...,selling surprising 39 main ​​40 m2 beautiful l...,selling surprising main beautiful location dis...
3,Need to sell office building with room on the ...,cần bán tòa văn phòng kết hợp phòng ở tầng trê...,need to sell office building with room on the ...,need to sell office building with room on the ...,need sell office building room upper floor fac...,need sell office building room upper facing ly...,need sell office building room upper facing st...
4,"4-storey house, restaurant business, busy busi...","+ nhà 4 tầng, kinh doanh hàng ăn, phố kinh doa...","4-storey house, restaurant business, busy busi...",4storey house restaurant business busy busines...,4storey house restaurant business busy busines...,4storey restaurant business busy business rent...,restaurant business busy business rented milli...


In [16]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

eng_df["text_wo_stopfreqrare"] = eng_df["text_wo_stopfreqrare"].apply(remove_urls)

In [17]:
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)


eng_df["text_wo_stopfreqrare"] = eng_df["text_wo_stopfreqrare"].apply(remove_html)

In [18]:
eng_df['text_wo_stopfreqrare']

0        selling service land hectares hanoi corner lot...
1        need sell townhouse adjacent bowl adjacent hou...
2        selling surprising main beautiful location dis...
3        need sell office building room upper facing st...
4        restaurant business busy business rented milli...
                               ...                        
26995    owner needs capital project abroad urgently se...
26996    solidly built dedicated living genuine wooden ...
26997    fold right mt two townhouse description locate...
26998    ward district ground bedrooms bathrooms worshi...
26999    divided lot two ladies car negotiable location...
Name: text_wo_stopfreqrare, Length: 167000, dtype: object

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
from sklearn.model_selection import train_test_split


data_train, data_test = train_test_split(eng_df['text_wo_stopfreqrare'], test_size=0.2, random_state=42)

In [21]:
training_corpus = data_train.values

vectorizer = CountVectorizer(min_df=3)

vectorizer.fit(training_corpus)
num_words = len(vectorizer.vocabulary_)
print("Number of distinct words to be used as features:",num_words)

Number of distinct words to be used as features: 14995


In [22]:
vectorizer.vocabulary_

{'owner': 9450,
 'need': 8799,
 'money': 8480,
 'urgently': 14008,
 'sell': 11746,
 'facing': 4820,
 'district': 3639,
 'spacious': 12335,
 'square': 12458,
 'structure': 12717,
 'basements': 908,
 'rooms': 11357,
 'income': 6670,
 'million': 8329,
 'pink': 9952,
 'home': 6220,
 'borrow': 1364,
 'bank': 861,
 'urgent': 14007,
 'selling': 11753,
 'little': 7676,
 'negotiation': 8839,
 'youngest': 14908,
 'bedrooms': 1020,
 'bathrooms': 935,
 'furniture': 5394,
 'handed': 5867,
 'investor': 6962,
 'comfortable': 2533,
 'river': 11272,
 'view': 14206,
 'chill': 2257,
 'apartment': 469,
 'many': 8016,
 'amenities': 351,
 'swimming': 12967,
 'pool': 10131,
 'gym': 5797,
 'bbq': 971,
 'childrens': 2255,
 'full': 5345,
 'international': 6908,
 'schools': 11596,
 'bis': 1238,
 'ims': 6640,
 'foreigners': 5187,
 'gathering': 5470,
 'live': 7680,
 'easy': 4251,
 'rent': 11026,
 'restaurants': 11170,
 'near': 8785,
 'markets': 8050,
 'supermarkets': 12878,
 'needs': 8809,
 'land': 7345,
 'village